In [ ]:
from landlab import RasterModelGrid, imshow_grid
from landlab.components import FlowAccumulator, GravelBedrockEroder
import numpy as np


grid = RasterModelGrid((100, 100), xy_spacing=25.0)
elev = grid.add_zeros("topographic__elevation", at="node")
# set constant random seed for consistent topographic roughness
np.random.seed(seed=5000)
# add topographic roughness
random_noise = (
    np.random.rand(len(grid.node_y)) / 1000.0
)  # impose topography values on model grid
grid["node"]["topographic__elevation"] += random_noise

sed = grid.add_zeros("soil__depth", at="node")
sed[4] = 300.0
grid.status_at_node[grid.perimeter_nodes] = grid.BC_NODE_IS_CLOSED
grid.status_at_node[5] = grid.BC_NODE_IS_FIXED_VALUE
fa = FlowAccumulator(grid, runoff_rate=10.0)
fa.run_one_step()
eroder = GravelBedrockEroder(grid, abrasion_coefficient=0.0005)
rock_elev = grid.at_node["bedrock__elevation"]


for _ in range(2000):
    rock_elev[grid.core_nodes] += 1.0
    elev[grid.core_nodes] += 1.0
    fa.run_one_step()
    eroder.run_one_step(100000.0)

int(elev[4] * 100)


In [ ]:
imshow_grid(grid, elev, cmap="terrain")